#こっちのNote参照
https://note.com/jnot/n/nd2c9469e4fd8

In [12]:
pip install graphillion

In [13]:
from graphillion import GraphSet as gs
import graphillion.tutorial as tl
import gc
import pandas as pd
import time
import itertools

#data import

In [14]:
#read Railway section data[s_sta,e_sta,dist]
df_list_tmp = pd.read_csv("/content/drive/MyDrive/2021_Vienna/Longest Route/Data/Austria_Edges_v10.csv", sep=',')

df_list = df_list_tmp[df_list_tmp["in_use"] == 1]

In [15]:
df_list

,s_sta,e_sta,dist,in_use
0,Lochau-Hoerbranz,Sud(Lauterach),8.0,1
1,Lustenau Markt,Sud(Lauterach),6.6,1
2,Sud(Lauterach),Feldkirch,32.2,1
3,Feldkirch,Tisis,7.3,1
4,Bludenz,Schruns,12.7,1
...,...,...,...,...
161,Ganserndorf,Bernhardsthal,43.9,1
162,Stockerau,Floridsdorf,21.6,1
163,Stockerau,Retz,55.6,1
164,Retz,Drosendorf,39.8,1


create list of end/start stations

In [16]:
start_end_station_set = set.union(set(df_list['s_sta']),set(df_list['e_sta']))
print(len(start_end_station_set))
print(start_end_station_set)

df_start_end_station = pd.DataFrame(
    data={'start_set': list(start_end_station_set), 
          'end_set': list(start_end_station_set)
    }
) 

141
{'Grunau-im-almtal', 'Kufstein', 'Steindorf', 'Brenner', 'Sollenau', 'Puchberg', 'Woergl', 'Tisis', 'Bernhardsthal', 'Villach', 'Gutenstein', 'VillachWarmbad', 'St.Veit-an-der-Glan', 'Neusiedl-am-see', 'Bruck-an-der-mur', 'St.pantaleon', 'Gleisdorf', 'Feldkirch', 'Thoeri-Maglern', 'Niederspaching', 'Scharding', 'Flughafen', 'Peuerbach', 'Ganserndorf', 'Sigmundsherberg', 'Waidhofen', 'Wolfsthal', 'Rennweg', 'Neumarkt-Kallham', 'Unterretzbach', 'Penzing', 'Gmunden', 'Pamhagen', 'Braunau-am-inn', 'Wetmannstaetten', 'Waidhofen-pastalozzi', 'Kledering', 'Drosendorf', 'St.michael', 'Voeklabruck', 'Emmersdorf', 'Heidenreichstein', 'Selzthal', 'Mariazell', 'Gmund', 'Schwarzach-St.Veit', 'Retz', 'Floridsdorf', 'Lamprechtshausen', 'Mogersdorf', 'Absdorf-hippersdorf', 'Burmoos', 'Sud(Lauterach)', 'Meidling', 'Tullun-stadt', 'Lochau-Hoerbranz', 'Krems-donau', 'Hutteldorf', 'Kastenreith', 'Traisen', 'Kittsee', 'Stockerau', 'Rosenbach', 'Summerau', 'Herzogenburg', 'Feldbach', 'St.Valentin', 'St.

In [17]:
univ = []
weights = {}

for i, v in df_list.iterrows():
   edge = (v['s_sta'], v['e_sta'])
   univ.append(edge)
   weights[edge] = v['dist']

gs.set_universe(univ)

#Main calculation 

In [18]:
AAA = []
BBB = []

#define start and end points
for i, v in df_start_end_station.iterrows():
   sta_s=(v['start_set'])
   sta_e=(v['end_set'])
   AAA.append(sta_s)
   BBB.append(sta_e)


 #計算してキロを返す 
def route_calc(start,end):
 #↓これでs-eの経路が列挙される
  paths = gs.paths(start, end)
  try:
    for path in paths.max_iter(weights):
      max_path = path
      #print(path)
      break
    distance = 0
    for edge in max_path:
      distance += weights[edge]
    return distance

  except: 
    "StopIteration"
    return "NA"

#メイン処理 
i_dst=0.0
j=1
time_sta=time.perf_counter()
longest_temp = 0
for A in AAA:
 for B in BBB:
   start=A
   end = B
   if(start != end ):
     i=route_calc(start,end)
     time_end=time.perf_counter()
     tim=time_end - time_sta
     if i != "NA":
       if i >= longest_temp:
        print("処理"+str(j)+"回目,"+start+","+end+",距離,"+str(i)+","+str(tim))
        longest_temp = i
     else:
       print("処理"+str(j)+"回目,"+start+","+end+",距離NA,"+str(tim))

     j=j+1
 BBB.pop(0)

処理1回目,Grunau-im-almtal,Kufstein,距離,2225.3999999999996,0.034786801999871386
処理3回目,Grunau-im-almtal,Brenner,距離,2309.3999999999996,0.085464376001255
処理7回目,Grunau-im-almtal,Tisis,距離,2435.4,0.1746473330003937
処理52回目,Grunau-im-almtal,Sud(Lauterach),距離,2460.3,1.6987697149997985
処理55回目,Grunau-im-almtal,Lochau-Hoerbranz,距離,2468.3,1.745015971000612


#write down all the routes over 2.350 km

In [19]:
define_long_thres = 2350

AAA = []
BBB = []

#define start and end points
for i, v in df_start_end_station.iterrows():
   sta_s=(v['start_set'])
   sta_e=(v['end_set'])
   AAA.append(sta_s)
   BBB.append(sta_e)
j = 1
for A in AAA:
 for B in BBB:
   start=A
   end = B
   if(start != end ):
     i=route_calc(start,end)
     time_end=time.perf_counter()
     tim=time_end - time_sta
     if i != "NA":
       if i >= define_long_thres:
        print("処理"+str(j)+"回目,"+start+","+end+",距離,"+str(i)+","+str(tim))
     j=j+1
 BBB.pop(0)
 

処理7回目,Grunau-im-almtal,Tisis,距離,2435.4,161.28475803000038
処理17回目,Grunau-im-almtal,Feldkirch,距離,2428.1000000000004,161.45523323900125
処理52回目,Grunau-im-almtal,Sud(Lauterach),距離,2460.3,162.7120826290011
処理55回目,Grunau-im-almtal,Lochau-Hoerbranz,距離,2468.3,162.76814422200005
処理101回目,Grunau-im-almtal,Schruns,距離,2420.0000000000005,163.5322697520005
処理121回目,Grunau-im-almtal,Lustenau Markt,距離,2466.9,163.83464031900075
処理130回目,Grunau-im-almtal,Bludenz,距離,2407.3,163.95826736600065
処理332回目,Steindorf,Lochau-Hoerbranz,距離,2353.0,175.803222165001
処理398回目,Steindorf,Lustenau Markt,距離,2351.6,184.60756470100023
処理971回目,Tisis,Niederspaching,距離,2361.9,195.0401658990013
処理972回目,Tisis,Scharding,距離,2384.7,195.0561791270011
処理974回目,Tisis,Peuerbach,距離,2365.5,195.08251216700046
処理980回目,Tisis,Neumarkt-Kallham,距離,2352.7,195.16313968600116
処理983回目,Tisis,Gmunden,距離,2408.4,195.2034763190004
処理985回目,Tisis,Braunau-am-inn,距離,2358.1000000000004,195.2302453299999
処理991回目,Tisis,Voeklabruck,距離,2366.6,195.32795163400078
処理1000

#Extract the longest route

In [20]:
start_sta = "Lochau-Hoerbranz"
goal_sta = "Grunau-im-almtal"
paths = gs.paths(start_sta, goal_sta)
for path in paths.max_iter(weights):
  max_path = path
  print(path)
  break

[('Absdorf-hippersdorf', 'Stockerau'), ('Amstetten', 'Poechlarn'), ('Amstetten', 'St.Valentin'), ('Attnang-Puchheim', 'Stainach-irdning'), ('Bischofshofen', 'Salzburg'), ('Bischofshofen', 'Stainach-irdning'), ('Bludenz', 'Innsbruck'), ('Bruck-an-der-mur', 'Graz'), ('Ebenfurth', 'Wulkaprodersdorf'), ('Fehring', 'Wr-Neustadt'), ('Feldbach', 'Fehring'), ('Feldkirch', 'Bludenz'), ('Floridsdorf', 'Leopoldau'), ('Ganserndorf', 'Leopoldau'), ('Ganserndorf', 'Marchegg'), ('Gleisdorf', 'Feldbach'), ('Graz', 'Gleisdorf'), ('Hadersdorf', 'Sigmundsherberg'), ('Handelskai', 'Heiligenstadt'), ('Handelskai', 'Rennweg'), ('Hbf', 'Flughafen'), ('Hbf', 'Marchegg'), ('Herzogenburg', 'Krems-donau'), ('Herzogenburg', 'Tullnerfeld'), ('Hutteldorf', 'Penzing'), ('Innsbruck', 'Jenbach'), ('Jenbach', 'Woergl'), ('Kastenreith', 'St.Valentin'), ('Klagenfurt', 'Rosenbach'), ('Kledering', 'Sollenau'), ('Krems-donau', 'Hadersdorf'), ('Lambach', 'Attnang-Puchheim'), ('Lambach', 'Wels'), ('Leobersdorf', 'Meidling'), 

In [21]:
start_temp = start_sta
print(start_sta)
for i in range(len(max_path)):
  for start_end_pair in max_path:
    if start_sta == start_end_pair[0]:
      print("->",start_end_pair[1],end='')
      start_sta = start_end_pair[1]
      max_path.remove(start_end_pair)
      break
    elif start_sta == start_end_pair[1]:
      print("->",start_end_pair[0],end='')
      start_sta = start_end_pair[0]
      max_path.remove(start_end_pair)
      break

Lochau-Hoerbranz
-> Sud(Lauterach)-> Feldkirch-> Bludenz-> Innsbruck-> Jenbach-> Woergl-> Zelamsee-> Schwarzach-St.Veit-> Spittal-milstaettersee-> Villach-> VillachWarmbad-> Rosenbach-> Klagenfurt-> St.Veit-an-der-Glan-> Unzmarkt-> St.michael-> Bruck-an-der-mur-> Graz-> Gleisdorf-> Feldbach-> Fehring-> Wr-Neustadt-> Ebenfurth-> Wulkaprodersdorf-> Neusiedl-am-see-> Parndorf-Ort-> Kledering-> Sollenau-> Leobersdorf-> Meidling-> Tullnerfeld-> Herzogenburg-> Krems-donau-> Hadersdorf-> Sigmundsherberg-> Absdorf-hippersdorf-> Stockerau-> Floridsdorf-> Leopoldau-> Ganserndorf-> Marchegg-> Hbf-> Flughafen-> Rennweg-> Handelskai-> Heiligenstadt-> Penzing-> Hutteldorf-> St.polten-> Poechlarn-> Amstetten-> St.Valentin-> Kastenreith-> Selzthal-> Linz-> Niederspaching-> Neumarkt-Kallham-> Scharding-> Ried-im-innkreis-> Braunau-am-inn-> Steindorf-> Salzburg-> Bischofshofen-> Stainach-irdning-> Attnang-Puchheim-> Lambach-> Wels-> Grunau-im-almtal

#type L??

In [ ]:
start_sta = "Lochau-Hoerbranz"
goal_sta = "Kammer-schoerfling"
paths = gs.paths(start_sta, goal_sta)
for path in paths.max_iter(weights):
  max_path = path
  print(path)
  break
start_temp = start_sta
print(start_sta)
for i in range(len(max_path)):
  for start_end_pair in max_path:
    if start_sta == start_end_pair[0]:
      print("->",start_end_pair[1],end='')
      start_sta = start_end_pair[1]
      max_path.remove(start_end_pair)
      break
    elif start_sta == start_end_pair[1]:
      print("->",start_end_pair[0],end='')
      start_sta = start_end_pair[0]
      max_path.remove(start_end_pair)
      break

[('Absdorf-hippersdorf', 'Stockerau'), ('Amstetten', 'Poechlarn'), ('Amstetten', 'St.Valentin'), ('Attnang-Puchheim', 'Stainach-irdning'), ('Bischofshofen', 'Salzburg'), ('Bischofshofen', 'Stainach-irdning'), ('Bludenz', 'Innsbruck'), ('Bruck-an-der-mur', 'Graz'), ('Ebenfurth', 'Wulkaprodersdorf'), ('Fehring', 'Wr-Neustadt'), ('Feldbach', 'Fehring'), ('Feldkirch', 'Bludenz'), ('Floridsdorf', 'Leopoldau'), ('Ganserndorf', 'Leopoldau'), ('Ganserndorf', 'Marchegg'), ('Gleisdorf', 'Feldbach'), ('Graz', 'Gleisdorf'), ('Hadersdorf', 'Sigmundsherberg'), ('Handelskai', 'Heiligenstadt'), ('Handelskai', 'Rennweg'), ('Hbf', 'Flughafen'), ('Hbf', 'Marchegg'), ('Herzogenburg', 'Krems-donau'), ('Herzogenburg', 'Tullnerfeld'), ('Hutteldorf', 'Penzing'), ('Innsbruck', 'Jenbach'), ('Jenbach', 'Woergl'), ('Kastenreith', 'St.Valentin'), ('Klagenfurt', 'Rosenbach'), ('Kledering', 'Sollenau'), ('Krems-donau', 'Hadersdorf'), ('Lambach', 'Attnang-Puchheim'), ('Lambach', 'Wels'), ('Leobersdorf', 'Meidling'), 

#Miscellaneous

In [ ]:
 #計算してキロを返す 
def route_calc_withpath(start,end):
 #↓これでs-eの経路が列挙される
  paths = gs.paths(start, end)
  for path in paths.max_iter(weights):
    max_path = path
      #print(path)
    break
  distance = 0
    for edge in max_path:
      distance += weights[edge]
    return distance

def route_print(path, search, end, station_dict, route_list):
    another = ""
    for i in path:
        if search in i:
            if search == i[0]:
                station = get_keys_from_value(station_dict, i[0])[0]
            else:
                station = get_keys_from_value(station_dict, i[1])[0]
            if re.search("[0-9]", station[-1]) == None:
                route_list.append(station)
            route_list_old = list(i)
            path.remove(i)
            route_list_old.remove(search)
            another = route_list_old[0]
            break
        else:
            another = search
            continue
    if another != "":
        route_print(path, another, end, station_dict, route_list)
    return route_list



if __name__ == '__main__':
    start = "Lochau-Hoerbranz"# 出発駅
    end = "Koeschach-Mauthen" # 終着駅
#    filename = "/content/drive/MyDrive/2021_Vienna/Longest Route/Data/Austria_Edges_Graphillion.csv"# 読み込みたいファイル名
    filename = "/content/drive/MyDrive/2021_Vienna/Longest Route/Data/Austria_Edges_v2.csv"# 読み込みたいファイル名
    station_dict = dict_create(filename)
    print(station_dict)
    route, distance, s_number, e_number = set_graph(start, end, station_dict, filename)
    creat_list = route_print(route, s_number, e_number, station_dict, [])
    end_station = [k for k, v in station_dict.items() if v == e_number]
    if re.search("[0-9]", end_station[0][-1]) != None:
        creat_list.append(end_station[0][:-1])
    else:
        creat_list.append(end_station[0])
    most_longest_route = "→".join(creat_list)
    print(most_longest_route)
    print(f"総距離{distance}km")
longest_start = 


#Miscellaneous

図示にトライしたが失敗

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

#nx.draw(gs.choice())
nx.draw(gs.choice(univ))                                                                                                                                                      
plt.show()

もともとのコード

In [ ]:
from graphillion import GraphSet as gs
import gc
import pandas as pd
import time
import itertools

time_sta=time.perf_counter()
 #路線データJR .csvを読み込む[s_sta,e_sta,kiro2]
df_list = pd.read_csv('/content/JR.csv', sep=',')
 #JRE .csvの駅名一覧を並べたもの[start_set,end_set]
df_list2=pd.read_csv('/content/order_set.csv', sep=',')

univ = []
weights = {}
AAA = []
BBB = []
 #JRE .csvを読み込む
for i, v in df_list.iterrows():
   edge = (v['s_sta'], v['e_sta'])
   univ.append(edge)
   weights[edge] = v['kiro2']
 #データをグラフにセット 
gs.set_universe(univ)

for i, v in df_list2.iterrows():
   sta_s=(v['start_set'])
   sta_e=(v['end_set'])
   AAA.append(sta_s)
   BBB.append(sta_e)
 #計算してキロを返す 
def root_calc(start,end):
 #↓これでs-eの経路が列挙される
 paths = gs.paths(start, end)
 max_path = next(paths.max_iter(weights))
 distance = 0
 for edge in max_path:
   distance += weights[edge]
 #2020 /6/12追記 メモリがすぐ足らなくなる問題を修正
 del paths
 gs.collect()
 return distance
 #ルート詳細を表示する際にこちらを使用  #def  root_print(path, search, end):
#   another = ""
#   for i in path:
#       if search in i:
#           root_list = list(i)
#           print(root_list)
#           path.remove(i)
#           root_list.remove(search)
#           another = root_list[0]
#           break
#       else:
#           another = search
#           continue
#   if another != "":
#       root_print(path, another, end)

 #メイン処理 
i_dst=0.0
j=1
for A in AAA:
 for B in BBB:
   start=A
   end = B
   if(start != end ):
     i=root_calc(start,end)
     time_end=time.perf_counter()
     tim=time_end - time_sta
     print("処理"+str(j)+"回目,"+start+","+end+",距離,"+str(i)+","+str(tim))
     j=j+1
 BBB.pop(0)

###prep

In [ ]:
df_list_temp = pd.read_csv("/content/drive/MyDrive/2021_Vienna/Longest Route/Data/test.csv", sep=',')

univ_temp = []
weights_temp = {}

for i_temp, v_temp in df_list_temp.iterrows():
   edge_temp = (v_temp['s_sta'], v_temp['e_sta'])
   univ_temp.append(edge_temp)
   weights_temp[edge_temp] = v_temp['dist']

gs.set_universe(univ_temp)